In [ ]:
import numpy as np
import time

from torch.autograd import Variable
from torchvision.utils import save_image, make_grid

import torch

import networks
from datasets import ImageDataset

from plotAllLosses import plotLosses, avg_loss

In [ ]:
def seperateImgs(metaData):
    if len(metaData.shape) == 4:
        return metaData[:,0:3]
    elif len(metaData.shape) == 3:
        return metaData[0:3]
    else:
        print("Cannot seperate images from a tensor with shape of " + str(metaData.shape))
def seperateMetas(metaData):
    if len(metaData.shape) == 4:
        return metaData[:,3:8]
    elif len(metaData.shape) == 3:
        return metaData[3:8]
    else:
        print("Cannot seperate metas from a tensor with shape of " + str(metaData.shape))

In [ ]:
def generatorPath(epoch):
    return saved_model_path + model_name + ("_generator_%d.pth" % epoch)
def discriminatorPath(epoch):
    return saved_model_path + model_name + ("_discriminator_%d.pth" % epoch)
def baseGeneratorPath():
    return saved_model_path + srgan_name + ("_generator.pth")
def autoencoderPath():
    return saved_model_path + encoder_name + (".pth")


In [ ]:
def save_sample(imgs, epoch):
    batch_size = 8 if imgs[0].shape[0] > 8 else imgs[0].shape[0]
    grid_imgs = []
    for img_seq in imgs:
        grid_imgs.append(make_grid(img_seq[0:batch_size], nrow=1, normalize=False))
    imgs_all = torch.cat(grid_imgs, dim=-1)
    save_image(imgs_all, sample_path + model_name + ("_%d.png" % epoch))

In [ ]:
cuda = torch.cuda.is_available()

hr_shape       = (256,256)
dataset_root   = "../Dataset/"
trainset_name  = "QuickRender_train/"
testset_name   = "QuickRender_test/"
batch_size     = 16
starting_epoch = 0
epochs         = 200

saved_model_path = "../Checkpoints/"
model_name       = "MetaSRGAN"
encoder_name     = "DNO_Autoencoder"
srgan_name       = "SRGAN"

sample_path = "../Samples/TrainingSamples/"

In [ ]:
generator         = networks.GeneratorResNet()
baseGenerator     = networks.GeneratorSlowNet()
discriminator     = networks.Discriminator(input_shape=(3, *hr_shape))
feature_extractor = networks.FeatureExtractor()
encoder           = networks.MetaAutoencoderTail()

In [ ]:
if starting_epoch > 0:
    generator.load_state_dict(torch.load(generatorPath(starting_epoch)))
    discriminator.load_state_dict(torch.load(discriminatorPath(starting_epoch)))
encoder.load_state_dict(torch.load(autoencoderPath()))
encoder.eval()
baseGenerator.load_state_dict(torch.load(baseGeneratorPath()))
baseGenerator.eval()
feature_extractor.eval()

In [ ]:
#Loss functions
criterion_GAN = torch.nn.MSELoss()
criterion_content = torch.nn.L1Loss()

In [ ]:
if cuda:
    print("Training on GPU")
    generator = generator.cuda()
    discriminator = discriminator.cuda()
    baseGenerator = baseGenerator.cuda()
    encoder = encoder.cuda()
    feature_extractor = feature_extractor.cuda()
    criterion_GAN = criterion_GAN.cuda()
    criterion_content = criterion_content.cuda()
    
Tensor = torch.cuda.FloatTensor if cuda else torch.Tensor

In [ ]:
#optimizers
learning_rate = 0.0002
optimizer_G  = torch.optim.Adam(generator.parameters(), lr=learning_rate, betas=(0.5, 0.999))
optimizer_D  = torch.optim.Adam(discriminator.parameters(), lr=learning_rate, betas=(0.5, 0.999))

In [ ]:
train_loader = torch.utils.data.DataLoader(
    dataset=ImageDataset(dataset_root + trainset_name, hr_shape), 
    batch_size=batch_size, 
    shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    dataset=ImageDataset(dataset_root + testset_name, hr_shape), 
    batch_size=batch_size, 
    shuffle=True
)

In [ ]:
generatorTrainLosses = []
generatorTestLosses  = []
discriminatorTrainLosses = []
discriminatorTestLosses  = []
generatorDeltas = []
discriminatorDeltas = []

lossLabels = ["Generator Train", "Generator Test", "Generator Delta"]

start_time = time.time()

depth, hr_depth, normal, hr_normal, obj, hr_obj, hr_imgs, hr_inf = None, None, None, None, None, None, None, None

for epoch in range(starting_epoch, epochs):
    torch.set_grad_enabled(True)
    holdGenLosses     = []
    holdDisLosses     = []
    for i, imgs in enumerate(train_loader):
        lr_imgs = Variable(seperateImgs(imgs["lr"]).type(Tensor))
        hr_imgs = Variable(seperateImgs(imgs["hr"]).type(Tensor))
        lr_metas = Variable(seperateMetas(imgs["lr"]).type(Tensor))
        hr_metas = Variable(seperateMetas(imgs["hr"]).type(Tensor))
        
        # Adversarial ground truths
        valid = Variable(Tensor(np.ones((lr_imgs.size(0), *discriminator.output_shape))), requires_grad=False)
        fake = Variable(Tensor(np.zeros((lr_imgs.size(0), *discriminator.output_shape))), requires_grad=False)
        
        depth  = lr_metas[:, 0:1]
        normal = lr_metas[:, 1:4]
        obj    = lr_metas[:, 4:5]
        b, c, h, w = lr_metas.shape
        
        depth  = torch.cat((depth, depth, depth), 1)
        obj    = torch.cat((obj, obj, obj), 1)
        
        depth  = baseGenerator(depth)
        normal = baseGenerator(normal)
        obj    = baseGenerator(obj)
        
        hr_depth  = hr_metas[:, 0:1]
        hr_normal = hr_metas[:, 1:4]
        hr_obj    = hr_metas[:, 4:5]
        
        hr_depth  = torch.cat((hr_depth, hr_depth, hr_depth), 1)
        hr_obj    = torch.cat((hr_obj, hr_obj, hr_obj), 1)
        
        # =============== construct metas =======================
        depth = depth[:, 0:1]
        obj   = obj[:, 0:1]
        hr_metas = torch.cat((depth, normal, obj), 1)
        
        meta_vec = encoder(hr_metas)
        
        #========== Train Generator =====================
        optimizer_G.zero_grad()
        #Inference
        hr_inf = generator(lr_imgs, meta_vec)
        #Adversarial loss
        loss_GAN = criterion_GAN(discriminator(hr_inf), valid)
        #Content loss
        inf_features  = feature_extractor(hr_inf)
        real_features = feature_extractor(hr_imgs)
        loss_content  = criterion_content(inf_features, real_features.detach())
        #total loss
        loss_G = loss_content + 1e-3 * loss_GAN
        loss_G.backward()
        optimizer_G.step()
        
        #========= Train Discriminator ==================
        optimizer_D.zero_grad()
        #loss of real and fake images
        loss_real = criterion_GAN(discriminator(hr_imgs), valid)
        loss_fake = criterion_GAN(discriminator(hr_inf.detach()), fake)
        #total loss
        loss_D = (loss_real + loss_fake) / 2
        loss_D.backward()
        optimizer_D.step()
        
        holdGenLosses.append(loss_G.item())
        holdDisLosses.append(loss_D.item())
    
    if epoch > 0:
        generatorTrainLosses.append(avg_loss(holdGenLosses))
        discriminatorTrainLosses.append(avg_loss(holdDisLosses))
    
    torch.save(generator.state_dict(), generatorPath(epoch+1))
    torch.save(discriminator.state_dict(), discriminatorPath(epoch+1))
    
    save_sample([depth, hr_depth, normal, hr_normal, obj, hr_obj, hr_inf, hr_imgs], epoch + 1)
    
    torch.set_grad_enabled(False)
    holdGenLosses     = []
    holdSlowGenLosses = []
    for i, imgs in enumerate(test_loader):
        lr_imgs = Variable(seperateImgs(imgs["lr"]).type(Tensor))
        hr_imgs = Variable(seperateImgs(imgs["hr"]).type(Tensor))
        lr_metas = Variable(seperateMetas(imgs["lr"]).type(Tensor))
        hr_metas = Variable(seperateMetas(imgs["hr"]).type(Tensor))
        
        # Adversarial ground truths
        valid = Variable(Tensor(np.ones((lr_imgs.size(0), *discriminator.output_shape))), requires_grad=False)
        fake = Variable(Tensor(np.zeros((lr_imgs.size(0), *discriminator.output_shape))), requires_grad=False)
        
        depth  = lr_metas[:, 0:1]
        normal = lr_metas[:, 1:4]
        obj    = lr_metas[:, 4:5]
        b, c, h, w = lr_metas.shape
        
        depth  = torch.cat((depth, depth, depth), 1)
        obj    = torch.cat((obj, obj, obj), 1)
        
        depth  = baseGenerator(depth)
        normal = baseGenerator(normal)
        obj    = baseGenerator(obj)
        
        #============= Construct Metas =======================
        depth = depth[:, 0:1]
        obj   = obj[:, 0:1]
        hr_metas = torch.cat((depth, normal, obj), 1)
        
        meta_vec = encoder(hr_metas)
        
        #========== Train Generator =====================
        optimizer_G.zero_grad()
        #Inference
        hr_inf = generator(lr_imgs, meta_vec)
        #Adversarial loss
        loss_GAN = criterion_GAN(discriminator(hr_inf), valid)
        #Content loss
        inf_features  = feature_extractor(hr_inf)
        real_features = feature_extractor(hr_imgs)
        loss_content  = criterion_content(inf_features, real_features.detach())
        #total loss
        loss_G = loss_content + 1e-3 * loss_GAN
        
        #========= Train Discriminator ==================
        #loss of real and fake images
        loss_real = criterion_GAN(discriminator(hr_imgs), valid)
        loss_fake = criterion_GAN(discriminator(hr_inf.detach()), fake)
        #total loss
        loss_D = (loss_real + loss_fake) / 2
        
        holdGenLosses.append(loss_G.item())
        holdDisLosses.append(loss_D.item())
    
    generatorTestLosses.append(avg_loss(holdGenLosses))
    discriminatorTestLosses.append(avg_loss(holdDisLosses))
    if epoch == 0:
        generatorTrainLosses.append(generatorTestLosses[-1])
        discriminatorTrainLosses.append(discriminatorTestLosses[-1])
    generatorDeltas.append(generatorTrainLosses[-1] - generatorTestLosses[-1])
    discriminatorDeltas.append(discriminatorTrainLosses[-1] - discriminatorTestLosses[-1])
    plotLosses(
        "MetaSRGAN",
        epoch+1,
        epochs,
        [generatorTrainLosses, generatorTestLosses, generatorDeltas],
        lossLabels,
        start_time
    )